In [11]:
import pandas as pd

sampling_rate = 2000  # Biopac EDA sampling rate in Hz

def load_behavioral(session_name, subject_id):
    filename = f"data/Behavioral_data/{session_name}Subject{subject_id}.csv"
    df = pd.read_csv(filename)
    
    # Clean and normalize n_back
    df['n_back'] = df['n_back'].str.strip().str.lower()
    
    def map_nback(text):
        if 'one back' in text:
            return '1_back'
        elif 'three back' in text:
            return '3_back'
        else:
            return None
    
    df['n_back_task'] = df['n_back'].apply(map_nback)
    
    # Calculate accuracy by trial and n_back_task
    df['correct'] = df['Response'] == df['Correct_Response']
    trial_summary = df.groupby(['TrialNumber', 'n_back_task']).agg(
        accuracy=('correct', 'mean'),
        mean_rt=('Response_Time', 'mean')
    ).reset_index()
    
    trial_summary['session'] = session_name.lower()
    
    return trial_summary

participants = ['3F', '4F', '6M', '8M', '11F']
all_data = []

for pid in participants:
    subject_num = ''.join(filter(str.isdigit, pid))
    gender = pid[-1]
    
    # Override gender for Biopac files if subject_num == '6' because files exist only for 6F
    biopac_gender = 'F' if subject_num == '6' else gender
    
    calming_behav = load_behavioral('Calming', subject_num)
    vexing_behav = load_behavioral('Vexing', subject_num)
    
    behavior_df = pd.concat([calming_behav, vexing_behav], ignore_index=True)
    
    eda_path = f"data/Biopac_data/EDA/Subject{subject_num}{biopac_gender}_EDA.csv"
    triggers_path = f"data/Biopac_data/Timing/Subject{subject_num}{biopac_gender}_Triggers_trial.csv"
    
    eda_df = pd.read_csv(eda_path, header=None, names=['EDA'])
    triggers_df = pd.read_csv(triggers_path)
    
    for idx, row in triggers_df.iterrows():
        for condition_prefix in ['calming', 'vexing']:
            for nback in ['1_back', '3_back']:
                start_col = f"{condition_prefix}_{nback}_start"
                end_col = f"{condition_prefix}_{nback}_end"
                
                if start_col in triggers_df.columns and end_col in triggers_df.columns:
                    start_time = row[start_col]
                    end_time = row[end_col]
                    
                    if pd.isna(start_time) or pd.isna(end_time):
                        continue
                    
                    start_idx = int(start_time * sampling_rate)
                    end_idx = int(end_time * sampling_rate)
                    
                    eda_slice = eda_df['EDA'].iloc[start_idx:end_idx]
                    mean_eda = eda_slice.mean()
                    
                    session_label = condition_prefix
                    nback_label = nback
                    
                    # TrialNumber in behavioral data might start at 1, triggers idx at 0, so +1 adjustment
                    behav_match = behavior_df[
                        (behavior_df['session'] == session_label) & 
                        (behavior_df['n_back_task'] == nback_label) & 
                        (behavior_df['TrialNumber'] == (idx + 1))
                    ]
                    
                    accuracy = behav_match['accuracy'].values[0] if not behav_match.empty else None
                    mean_rt = behav_match['mean_rt'].values[0] if not behav_match.empty else None
                    
                    all_data.append({
                        'participant': pid,
                        'trial': idx + 1,
                        'condition': f"{condition_prefix.capitalize()} {nback.replace('_', '-').capitalize()}",
                        'mean_eda': mean_eda,
                        'accuracy': accuracy,
                        'mean_rt': mean_rt
                    })

combined_df = pd.DataFrame(all_data)
print(combined_df.head())


FileNotFoundError: [Errno 2] No such file or directory: 'data/Biopac_data/Timing/Subject6M_Triggers_trial.csv'

In [8]:
print(eda_df.columns.tolist())


['6.6223']
